In [1]:
%%bash

echo Hello, try to find the environments before running
echo $(pwd)
echo $VIRTUAL_ENV
echo $SPARK_HOME

Hello, try to find the environments before running
/Users/kiang/Desktop/PythonProjects/Repos/ApacheSparkLearning/LearningSparkV2/chapter3/py/src
/Users/kiang/Desktop/PythonProjects/Repos/ApacheSparkLearning/sparklearning_env



# The *DataFrame* API

In [2]:
import os
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pathlib import Path 

In [3]:
spark = (SparkSession
       .builder
       .appName("Aggregations and Computations in Spark Exercise - Chapter 3")
       .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/13 21:58:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/13 21:58:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
sf_fire_datasets = "/Users/kiang/Desktop/PythonProjects/Repos/ApacheSparkLearning/LearningSparkV2/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"

In [5]:
#StructType Parameter with a List of StructField() methods per column.
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
              StructField('UnitID', StringType(), True),
              StructField('IncidentNumber', IntegerType(), True),
              StructField('CallType', StringType(), True),
              StructField('CallDate', StringType(), True),
              StructField('WatchDate', StringType(), True),
              StructField('CallFinalDisposition', StringType(), True),
              StructField('AvailableDtTm', StringType(), True),
              StructField('Address', StringType(), True),
              StructField('City', StringType(), True),
              StructField('Zipcode', IntegerType(), True),
              StructField('Battalion', StringType(), True),
              StructField('StationArea', StringType(), True),
              StructField('Box', StringType(), True),
              StructField('OriginalPriority', StringType(), True),
              StructField('Priority', StringType(), True),
              StructField('FinalPriority', IntegerType(), True),
              StructField('ALSUnit', BooleanType(), True),
              StructField('CallTypeGroup', StringType(), True),
              StructField('NumAlarms', IntegerType(), True),
              StructField('UnitType', StringType(), True),
              StructField('UnitSequenceInCallDispatch', IntegerType(), True),
              StructField('FirePreventionDistrict', StringType(), True),
              StructField('SupervisorDistrict', StringType(), True),
              StructField('Neighborhood', StringType(), True),
              StructField('Location', StringType(), True),
              StructField('RowID', StringType(), True),
              StructField('Delay', FloatType(), True)])

# Using the DataFrameReader Interface to read the CSV file into Spark.
fire_df = spark.read.csv(sf_fire_datasets, header = True, schema = fire_schema)
            

24/06/13 21:58:52 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Accessing Columns from spark dataframes

> To access columns, use ```.columns```
> You can access them individual columnnames by 
> ```.columns[index]```

> Columns are stored as a list, so you can iterate through it.
```for col in df.columns```

In [67]:
fire_df.columns
fire_df.columns[0]
for col in fire_df.columns:
    print(col)

CallNumber
UnitID
IncidentNumber
CallType
CallDate
WatchDate
CallFinalDisposition
AvailableDtTm
Address
City
Zipcode
Battalion
StationArea
Box
OriginalPriority
Priority
FinalPriority
ALSUnit
CallTypeGroup
NumAlarms
UnitType
UnitSequenceInCallDispatch
FirePreventionDistrict
SupervisorDistrict
Neighborhood
Location
RowID
Delay


In [7]:
fire_df.show()

24/06/13 21:59:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|      UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+

In [25]:
fire_df.printSchema()
# fire_df.createOrReplaceTempView("temp_tbl")

# spark.sql("""
        #   select * from temp_tbl limit 100
        #   """)

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [ ]:
#?SELECT
fire_df.select("CallNumber", "UnitID", "IncidentNumber").show(5, truncate = False )

#?GROUP BY, ORDER BY, SELECT, WHERE: Count the number of rows that are in each unique call type.
(fire_df.select("CallType")
            .where(col("CallType").isNotNull()) #*Is this step needed, does spark innately possess the ability to remove rows with null values?
            .groupBy("CallType")
            .count()
            .orderBy('count', ascending = False)
            .show(truncate = False)
        )

(fire_df.count())

+----------+------+--------------+
|CallNumber|UnitID|IncidentNumber|
+----------+------+--------------+
|20110016  |T13   |2003235       |
|20110022  |M17   |2003241       |
|20110023  |M41   |2003242       |
|20110032  |E11   |2003250       |
|20110043  |B04   |2003259       |
+----------+------+--------------+
only showing top 5 rows



+-------------------------------+------+
|CallType                       |count |
+-------------------------------+------+
|Medical Incident               |113794|
|Structure Fire                 |23319 |
|Alarms                         |19406 |
|Traffic Collision              |7013  |
|Citizen Assist / Service Call  |2524  |
|Other                          |2166  |
|Outside Fire                   |2094  |
|Vehicle Fire                   |854   |
|Gas Leak (Natural and LP Gases)|764   |
|Water Rescue                   |755   |
|Odor (Strange / Unknown)       |490   |
|Electrical Hazard              |482   |
|Elevator / Escalator Rescue    |453   |
|Smoke Investigation (Outside)  |391   |
|Fuel Spill                     |193   |
|HazMat                         |124   |
|Industrial Accidents           |94    |
|Explosion                      |89    |
|Train / Rail Incident          |57    |
|Aircraft Emergency             |36    |
+-------------------------------+------+
only showing top

175296

## Schemas in Spark Dataframes

### There are some benefits of declaring a schema early before reading in datasource. (Schema on Read)

> * You relieve Spark from the onus of inferring data types.
> 
> * You prevent spark from creating a separate job just to read a large portion of your file to ascertain the schema, which for a large data file can be expensive and time consuming.
> 
> * You can detect errors early if data doesn't match the schema.


### Two ways to define a Schema
> Spark allows you to define a schema in two ways. 
> * One is to define it programmatially, 
> 
> * The other is to employ a Data Definition Language (DDL) string, which is much simpler and easier to read.


To define a schema programmatically for a DataFrame with three named columns, author, title, and pages, you can use the Spark DataFrame API. 

For example:

In Scala
```
import org.apache.spark.sql.types._
val schema = StructType(Array(
                        StructField("author", StringType, false),
                        StructField("title", StringType, false), 
                        StructField("pages", IntegerType, false)
                        ))
```
In Python

```
from pyspark.sql.types import *
schema = StructType([StructField("author", StringType(), False),
      StructField("title", StringType(), False),
      StructField("pages", IntegerType(), False)])
```

Defining the same schema using DDL is much simpler:

In Scala:
```
val schema = "author STRING, title STRING, pages INT"
```

Python
```
schema = "author STRING, title STRING, pages INT"
```


#import org.apache.spark.sql.types#

Read up on this to get more information on how to use spark types.

In [ ]:
(fire_df.select(
                "CallNumber"
                ,"UnitID"
                ,"IncidentNumber"
                ,"CallType"
                ,"CallDate"
                ,"WatchDate"
                ,"CallFinalDisposition"
                ,"AvailableDtTm"
                ,"Address"
                ,"City"
                ,"Zipcode"
                ,"Battalion"
                ,"StationArea"
                ,"Box"
                ,"OriginalPriority"
                ,"Priority"
                ,"FinalPriority"
                ,"ALSUnit"
                ,"CallTypeGroup"
                ,"NumAlarms"
                ,"UnitType"
                ,"UnitSequenceInCallDispatch"
                ,"FirePreventionDistrict"
                ,"SupervisorDistrict"
                ,"Neighborhood"
                ,"Location"
                ,"RowID"
                ,"Delay"
                )
                .show(10)
                )

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

In [24]:
# Use .schema to retrieve the DDL from the declared df, if you want to reuse it.
print(fire_df.schema)

# Retrieve the inside of the field
print(fire_df.schema['UnitID'])

#Use this 
print(fire_df.printSchema())

StructType([StructField('CallNumber', IntegerType(), True), StructField('UnitID', StringType(), True), StructField('IncidentNumber', IntegerType(), True), StructField('CallType', StringType(), True), StructField('CallDate', StringType(), True), StructField('WatchDate', StringType(), True), StructField('CallFinalDisposition', StringType(), True), StructField('AvailableDtTm', StringType(), True), StructField('Address', StringType(), True), StructField('City', StringType(), True), StructField('Zipcode', IntegerType(), True), StructField('Battalion', StringType(), True), StructField('StationArea', StringType(), True), StructField('Box', StringType(), True), StructField('OriginalPriority', StringType(), True), StructField('Priority', StringType(), True), StructField('FinalPriority', IntegerType(), True), StructField('ALSUnit', BooleanType(), True), StructField('CallTypeGroup', StringType(), True), StructField('NumAlarms', IntegerType(), True), StructField('UnitType', StringType(), True), St

## Reading from JSON Files 

You can read the data from the json files too.
Try to read in the blogs.json file in the ./databricks-datasets

Note : blogs.json used here is not a typical JSON file. You'll likely run into problems when running with actual json files.
https://spark.apache.org/docs/latest/sql-data-sources-json.html

In [37]:
blogs_path  = "/Users/kiang/Desktop/PythonProjects/Repos/ApacheSparkLearning/LearningSparkV2/databricks-datasets/learning-spark-v2/blogs.json"

# Without defining, its schema on read.
blogs_df = spark.read.json(blogs_path)

blogs_df.show()

+--------------------+---------+-----+---+-------+---------+-----------------+
|           Campaigns|    First| Hits| Id|   Last|Published|              Url|
+--------------------+---------+-----+---+-------+---------+-----------------+
| [twitter, LinkedIn]|    Jules| 4535|  1|  Damji| 1/4/2016|https://tinyurl.1|
| [twitter, LinkedIn]|   Brooke| 8908|  2|  Wenig| 5/5/2018|https://tinyurl.2|
|[web, twitter, FB...|    Denny| 7659|  3|    Lee| 6/7/2019|https://tinyurl.3|
|       [twitter, FB]|Tathagata|10568|  4|    Das|5/12/2018|https://tinyurl.4|
|[web, twitter, FB...|    Matei|40578|  5|Zaharia|5/14/2014|https://tinyurl.5|
| [twitter, LinkedIn]|  Reynold|25568|  6|    Xin| 3/2/2015|https://tinyurl.6|
+--------------------+---------+-----+---+-------+---------+-----------------+



## Columns and Expressions (Page. 54)

You can also use logical or mathematical expressions on columns.

You could create using
```expr("columnName  * 5")``` or 
```(expr("columnName1 - 5") > col("columnName"))```
where colname is a spark type.


```expr()``` is a part of the pyspark.sql.functions (python) and org.apache.spark.sql.functions (Scala) packages

Note: Where do you use the expr() function? 
* You can use it inside the ```.select()``` statement i.e. ```df.select(expr())```. It will compute values base on the expression.
* Or you could use it inside the ```.where()``` i.e ```df.select().where(expr())``` statement.
* You can also use it within the ```.withColumn("Name" , expr("ColumnName * 2"))```

In [54]:
# Adds a boolean row.
(blogs_df.select('*'
                 ,expr("First == 'Jules'")
                 ,expr("Hits * 2")
                 ).show())

+--------------------+---------+-----+---+-------+---------+-----------------+---------------+----------+
|           Campaigns|    First| Hits| Id|   Last|Published|              Url|(First = Jules)|(Hits * 2)|
+--------------------+---------+-----+---+-------+---------+-----------------+---------------+----------+
| [twitter, LinkedIn]|    Jules| 4535|  1|  Damji| 1/4/2016|https://tinyurl.1|           true|      9070|
| [twitter, LinkedIn]|   Brooke| 8908|  2|  Wenig| 5/5/2018|https://tinyurl.2|          false|     17816|
|[web, twitter, FB...|    Denny| 7659|  3|    Lee| 6/7/2019|https://tinyurl.3|          false|     15318|
|       [twitter, FB]|Tathagata|10568|  4|    Das|5/12/2018|https://tinyurl.4|          false|     21136|
|[web, twitter, FB...|    Matei|40578|  5|Zaharia|5/14/2014|https://tinyurl.5|          false|     81156|
| [twitter, LinkedIn]|  Reynold|25568|  6|    Xin| 3/2/2015|https://tinyurl.6|          false|     51136|
+--------------------+---------+-----+---+----

In [52]:
# Use expression to filter a value.
blogs_df.select('*').where(expr("First == 'Jules'")).show()

+-------------------+-----+----+---+-----+---------+-----------------+
|          Campaigns|First|Hits| Id| Last|Published|              Url|
+-------------------+-----+----+---+-----+---------+-----------------+
|[twitter, LinkedIn]|Jules|4535|  1|Damji| 1/4/2016|https://tinyurl.1|
+-------------------+-----+----+---+-----+---------+-----------------+



In [91]:
print(blogs_df.printSchema())

# Notice Publish is a string. Because this table is schema on Read, spark inferred Published wrong, thus, order by will not work
# correctly on strings.
(blogs_df
 .withColumn('FullName', concat(expr("First") , expr("Last"))) # Used in withColumn
 .select('*')
 .orderBy("Hits")
 ).show()

root
 |-- Campaigns: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- First: string (nullable = true)
 |-- Hits: long (nullable = true)
 |-- Id: long (nullable = true)
 |-- Last: string (nullable = true)
 |-- Published: string (nullable = true)
 |-- Url: string (nullable = true)

None
+--------------------+---------+-----+---+-------+---------+-----------------+------------+
|           Campaigns|    First| Hits| Id|   Last|Published|              Url|    FullName|
+--------------------+---------+-----+---+-------+---------+-----------------+------------+
| [twitter, LinkedIn]|    Jules| 4535|  1|  Damji| 1/4/2016|https://tinyurl.1|  JulesDamji|
|[web, twitter, FB...|    Denny| 7659|  3|    Lee| 6/7/2019|https://tinyurl.3|    DennyLee|
| [twitter, LinkedIn]|   Brooke| 8908|  2|  Wenig| 5/5/2018|https://tinyurl.2| BrookeWenig|
|       [twitter, FB]|Tathagata|10568|  4|    Das|5/12/2018|https://tinyurl.4|TathagataDas|
| [twitter, LinkedIn]|  Reynold|25568|  6|

### Important Note:

> Column Objects in a Dataframe cannot exist in isolation (i.e. $"FullName") each column is a part of a row in a record and all the rows together constitute a Dataframe, which was we will learn later in the chapter,
> is really a Dataset[Row] in Scala.
>
> Next Pg 57. on Rows*** Code in Scala.

Will continue on to Dataframe Operations pg 58.